In [1]:
import os
import wandb
import pandas as pd

In [4]:
from dotenv import load_dotenv

load_dotenv()
WANDB_API_KEY = os.environ.get('WANDB_API_KEY')

wandb.login(key=WANDB_API_KEY)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: luquc (luquc-national-economics-university). Use `wandb login --relogin` to force relogin


True

In [45]:
df = pd.read_csv("D:/Mlops_Project_K64/bank-additional-full.csv", sep=";")
X = df.drop(columns=["y"])
y = df["y"]

In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

In [47]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [48]:
X_train_enc = pd.get_dummies(X_train, drop_first=True)
X_test_enc = pd.get_dummies(X_test, drop_first=True)

In [53]:
run = wandb.init(project="Bank-Marketing", job_type="data")

# Log artifact
artifact = wandb.Artifact("raw_data", type="dataset")
artifact.add_file("D:/Mlops_Project_K64/bank-additional-full.csv")
run.log_artifact(artifact)

run.finish()

In [ ]:
run = wandb.init(project="Bank-Marketing", job_type="training")

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import joblib

# Train model 1
model_rf = RandomForestClassifier()
model_rf.fit(X_train_enc, y_train)
wandb.log({"rf_accuracy": model_rf.score(X_test_enc, y_test)})

# Log model_rf as artifact (save it as a file in memory before uploading)
artifact_rf = wandb.Artifact("rf-model", type="model")
model_rf_filename = "model_rf.pkl"
joblib.dump(model_rf, model_rf_filename)  # Save the model as a .pkl file
artifact_rf.add_file(model_rf_filename)  # Add the saved model file to the artifact
run.log_artifact(artifact_rf)

# Train model 2
model_lr = LogisticRegression()
model_lr.fit(X_train_enc, y_train)
run.log({"lr_accuracy": model_lr.score(X_test_enc, y_test)})

# Log model_lr as artifact (save it as a file in memory before uploading)
artifact_lr = wandb.Artifact("lr-model", type="model")
model_lr_filename = "model_lr.pkl"
joblib.dump(model_lr, model_lr_filename)  # Save the model as a .pkl file
artifact_lr.add_file(model_lr_filename)  # Add the saved model file to the artifact
run.log_artifact(artifact_lr)

c:\Users\HP\anaconda3\envs\mushroom\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


<Artifact lr-model>

In [51]:
# # For versioning, you can append the current run or a version number
# import time
# artifact_rf = wandb.Artifact(f"rf-model-v{int(time.time())}", type="model")
# artifact_rf.add_file(model_rf_filename)  # Add the saved model file to the artifact
# wandb.log_artifact(artifact_rf)
